# The cross-nested logit model

## Specification of the utility functions

In [1]:
import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
import biogeme.models as models
import biogeme.optimization as opt
from biogeme.expressions import Beta, log
from scipy.stats import chi2
import biogeme.version as ver

Version of Biogeme

In [2]:
print(ver.getText())

biogeme 3.2.8 [2021-07-26]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
url = (
    'https://courses.edx.org/asset-v1:EPFLx+ChoiceModels2x+3T2021+type@asset+block@'
       'swissmetro.dat'
)
df = pd.read_csv(url, sep='\t')
database = db.Database('swissmetro', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Removing some observations
exclude = CHOICE == 0
database.remove(exclude)

# Dummy variables variables for segmentation
age_00_24 = AGE == 1
age_25_39 = AGE == 2
age_40_54 = AGE == 3
age_55_65 = AGE == 4
age_65_plus = AGE == 5


female = 1 - MALE
male = MALE
noGA = GA == 0

FIRST_CLASS = FIRST
SECOND_CLASS = FIRST == 0

# Parameters to be estimated
ASC_CAR_MALE = Beta('ASC_CAR_MALE', 0, None, None, 0)
ASC_CAR_FEMALE = Beta('ASC_CAR_FEMALE', 0, None, None, 0)
ASC_CAR = ASC_CAR_MALE * male + ASC_CAR_FEMALE * female

ASC_TRAIN_MALE = Beta('ASC_TRAIN_MALE', 0, None, None, 0)
ASC_TRAIN_FEMALE = Beta('ASC_TRAIN_FEMALE', 0, None, None, 0)
ASC_TRAIN = ASC_TRAIN_MALE * male + ASC_TRAIN_FEMALE * female

B_TIME_CAR = Beta('B_TIME_CAR', 0, None, None, 0)

B_TIME_TRAIN_GA = Beta('B_TIME_TRAIN_GA', 0, None, None, 0)
B_TIME_TRAIN_noGA = Beta('B_TIME_TRAIN_noGA', 0, None, None, 0)
B_TIME_TRAIN = B_TIME_TRAIN_GA * GA + B_TIME_TRAIN_noGA * noGA

B_TIME_SM_GA = Beta('B_TIME_SM_GA', 0, None, None, 0)
B_TIME_SM_noGA = Beta('B_TIME_SM_noGA', 0, None, None, 0)
B_TIME_SM = B_TIME_SM_GA * GA + B_TIME_SM_noGA * noGA

B_COST_CAR_FIRST = Beta('B_COST_CAR_FIRST', 0, None, None, 0)
B_COST_CAR_SECOND = Beta('B_COST_CAR_SECOND', 0, None, None, 0)
B_COST_CAR = B_COST_CAR_FIRST * FIRST_CLASS + B_COST_CAR_SECOND * SECOND_CLASS

B_COST_TRAIN_FIRST = Beta('B_COST_TRAIN_FIRST', 0, None, None, 0)
B_COST_TRAIN_SECOND = Beta('B_COST_TRAIN_SECOND', 0, None, None, 0)
B_COST_TRAIN = B_COST_TRAIN_FIRST * FIRST_CLASS + B_COST_TRAIN_SECOND * SECOND_CLASS

B_COST_SM_FIRST = Beta('B_COST_SM_FIRST', 0, None, None, 0)
B_COST_SM_SECOND = Beta('B_COST_SM_SECOND', 0, None, None, 0)
B_COST_SM = B_COST_SM_FIRST * FIRST_CLASS + B_COST_SM_SECOND * SECOND_CLASS

B_HEADWAY_TRAIN_00_24 = Beta('B_HEADWAY_TRAIN_00_24', 0, None, None, 0)
B_HEADWAY_TRAIN_25_39 = Beta('B_HEADWAY_TRAIN_25_39', 0, None, None, 0)
B_HEADWAY_TRAIN_40_54 = Beta('B_HEADWAY_TRAIN_40_54', 0, None, None, 0)
B_HEADWAY_TRAIN_55_65 = Beta('B_HEADWAY_TRAIN_55_65', 0, None, None, 0)
B_HEADWAY_TRAIN_65_plus = Beta('B_HEADWAY_TRAIN_65_plus', 0, None, None, 0)

B_HEADWAY_TRAIN = B_HEADWAY_TRAIN_00_24 * age_00_24 + \
    B_HEADWAY_TRAIN_25_39 * age_25_39 + \
    B_HEADWAY_TRAIN_40_54 * age_40_54 + \
    B_HEADWAY_TRAIN_55_65 * age_55_65 + \
    B_HEADWAY_TRAIN_65_plus * age_65_plus

B_HEADWAY_SM_00_24 = Beta('B_HEADWAY_SM_00_24', 0, None, None, 0)
B_HEADWAY_SM_25_39 = Beta('B_HEADWAY_SM_25_39', 0, None, None, 0)
B_HEADWAY_SM_40_54 = Beta('B_HEADWAY_SM_40_54', 0, None, None, 0)
B_HEADWAY_SM_55_65 = Beta('B_HEADWAY_SM_55_65', 0, None, None, 0)
B_HEADWAY_SM_65_plus = Beta('B_HEADWAY_SM_65_plus', 0, None, None, 0)

B_HEADWAY_SM = B_HEADWAY_SM_00_24 * age_00_24 + \
    B_HEADWAY_SM_25_39 * age_25_39 + \
    B_HEADWAY_SM_40_54 * age_40_54 + \
    B_HEADWAY_SM_55_65 * age_55_65 + \
    B_HEADWAY_SM_65_plus * age_65_plus

# Definition of new variables
SM_COST = SM_CO * (GA == 0)
TRAIN_COST = TRAIN_CO * (GA == 0)
TRAIN_TT_SCALED = TRAIN_TT / 60
TRAIN_COST_SCALED = TRAIN_COST / 100
SM_TT_SCALED = SM_TT / 60
SM_COST_SCALED = SM_COST / 100
CAR_TT_SCALED = CAR_TT / 60
CAR_COST_SCALED = CAR_CO / 100

TRAIN_HE_SCALED = TRAIN_HE / 60
SM_HE_SCALED = SM_HE / 60

def piecewise_cost(x):
    """
    Piecewise linear transformation of the variable
    """
    piecewiseVariables = models.piecewiseVariables(x, [0, 0.5, 1, 1.75, None])
    return (piecewiseVariables[0] +
            Beta('pw_cost_0.5_1', 0, None, None, 0) * piecewiseVariables[1] +
            Beta('pw_cost_1_1.75', 0, None, None, 0) * piecewiseVariables[2] +
            Beta('pw_cost_1.75_more', 0, None, None, 0) * piecewiseVariables[3])

# Definition of the utility functions
V1 = ASC_TRAIN + \
     B_TIME_TRAIN * log(TRAIN_TT_SCALED) + \
     B_COST_TRAIN * piecewise_cost(TRAIN_COST_SCALED) + \
     B_HEADWAY_TRAIN * TRAIN_HE**0.5

V2 = B_TIME_SM * log(SM_TT_SCALED) + \
     B_COST_SM * piecewise_cost(SM_COST_SCALED) + \
     B_HEADWAY_SM * SM_HE**0.5

V3 = ASC_CAR + \
     B_TIME_CAR * log(CAR_TT_SCALED) + \
     B_COST_CAR * piecewise_cost(CAR_COST_SCALED)

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives
av = {1: TRAIN_AV,
      2: SM_AV,
      3: CAR_AV}

## Nested logit: existing alternatives

In [4]:
MU = Beta('MU', 1, 0, None, 0)
existing = MU, [1, 3]
future = 1.0, [2]
nests = existing, future
logprob = models.lognested(V, av, nests, CHOICE)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'nested_existing'
nested_existing_results = biogeme.estimate(algorithm=opt.bioNewton)

In [5]:
print(nested_existing_results.shortSummary())

Results for model nested_existing
Nbr of parameters:		29
Sample size:			10719
Excluded data:			9
Final log likelihood:		-7640.153
Akaike Information Criterion:	15338.31
Bayesian Information Criterion:	15549.42



In [6]:
nested_existing_table = nested_existing_results.getEstimatedParameters()
nested_existing_table

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR_FEMALE,-0.990797,0.145150,-6.826006,8.731016e-12,0.156796,-6.319022,2.632246e-10
ASC_CAR_MALE,-0.797239,0.134885,-5.910497,3.410778e-09,0.142710,-5.586415,2.318049e-08
ASC_TRAIN_FEMALE,1.296326,0.175505,7.386280,1.509903e-13,0.179873,7.206912,5.724310e-13
ASC_TRAIN_MALE,0.703357,0.166891,4.214466,2.503702e-05,0.170435,4.126832,3.677945e-05
B_COST_CAR_FIRST,-1.322511,0.216515,-6.108171,1.007793e-09,0.220086,-6.009059,1.866031e-09
B_COST_CAR_SECOND,-0.722826,0.146262,-4.942000,7.732512e-07,0.143131,-5.050099,4.415809e-07
B_COST_SM_FIRST,-1.811400,0.301255,-6.012850,1.822895e-09,0.325482,-5.565279,2.617328e-08
B_COST_SM_SECOND,-1.694546,0.264149,-6.415107,1.407241e-10,0.285854,-5.928023,3.066037e-09
B_COST_TRAIN_FIRST,-1.902173,0.327262,-5.812381,6.159037e-09,0.359430,-5.292190,1.208604e-07
B_COST_TRAIN_SECOND,-1.485451,0.255247,-5.819660,5.896758e-09,0.278323,-5.337157,9.441514e-08


## Cross-nested logit

In [7]:
MU_EXISTING = Beta('MU_EXISTING', 1, None, None, 0)
MU_PUBLIC = Beta('MU_PUBLIC', 1, None, None, 0)

ALPHA_EXISTING = Beta('ALPHA_EXISTING', 0.5, 0, 1, 0)
ALPHA_PUBLIC = 1 - ALPHA_EXISTING

alpha_existing = {1: ALPHA_EXISTING,
                  2: 0.0,
                  3: 1.0}

alpha_public = {1: ALPHA_PUBLIC,
                2: 1.0,
                3: 0.0}

nest_existing = MU_EXISTING, alpha_existing
nest_public = MU_PUBLIC, alpha_public
nests = nest_existing, nest_public

# The choice model is a cross-nested logit, with availability conditions
logprob = models.logcnl_avail(V, av, nests, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'cnl'

# Estimate the parameters
cnl_results = biogeme.estimate(algorithm=opt.bioNewton)

In [8]:
print(cnl_results.shortSummary())

Results for model cnl
Nbr of parameters:		31
Sample size:			10719
Excluded data:			9
Final log likelihood:		-7631.122
Akaike Information Criterion:	15324.24
Bayesian Information Criterion:	15549.92



In [9]:
cnl_table = cnl_results.getEstimatedParameters()
cnl_table

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ALPHA_EXISTING,0.763782,0.025414,30.053008,0.000000e+00,0.024572,31.083238,0.000000e+00
ASC_CAR_FEMALE,-0.988084,0.142156,-6.950715,3.634426e-12,0.155343,-6.360648,2.009048e-10
ASC_CAR_MALE,-0.789063,0.132146,-5.971142,2.355983e-09,0.141928,-5.559596,2.703999e-08
ASC_TRAIN_FEMALE,1.427364,0.157722,9.049865,0.000000e+00,0.160929,8.869516,0.000000e+00
ASC_TRAIN_MALE,0.880467,0.151276,5.820270,5.875247e-09,0.152153,5.786717,7.177529e-09
B_COST_CAR_FIRST,-1.326807,0.214901,-6.174051,6.656222e-10,0.219086,-6.056098,1.394630e-09
B_COST_CAR_SECOND,-0.729064,0.146057,-4.991631,5.987169e-07,0.143643,-5.075533,3.864108e-07
B_COST_SM_FIRST,-1.820128,0.298930,-6.088806,1.137559e-09,0.324253,-5.613297,1.985069e-08
B_COST_SM_SECOND,-1.705978,0.262368,-6.502240,7.913292e-11,0.285530,-5.974768,2.304186e-09
B_COST_TRAIN_FIRST,-1.892259,0.321283,-5.889690,3.869208e-09,0.353509,-5.352783,8.661163e-08


The $\mu$ parameter of each nest is larger than 1, as requested by the theory. 

In [10]:
mu_existing = cnl_table.loc['MU_EXISTING','Value']
mu_existing

1.2119875154425226

In [11]:
mu_public = cnl_table.loc['MU_PUBLIC','Value']
mu_public

9.598975723037194

The $\alpha$ parameter is different from 0 and 1.

In [12]:
alpha = cnl_table.loc['ALPHA_EXISTING','Value']
alpha

0.7637816584134136

It is significantly different from 0.

In [13]:
cnl_table.loc['ALPHA_EXISTING','Rob. t-test']

31.083238463186497

It is significantly different from 1.

In [14]:
alpha_stderr = cnl_table.loc['ALPHA_EXISTING', 'Rob. Std err']
tested_value = 1
ttest = (tested_value - alpha) / alpha_stderr
ttest

9.613259182168076

We test the null hypothesis that the two models are equivalent using a likelihood ratio test: 

In [15]:
LL_nested_existing = nested_existing_results.data.logLike
LL_cnl = cnl_results.data.logLike
LR = -2 * (LL_nested_existing - LL_cnl)
LR

18.060659332759315

Number of degrees of freedom:

In [16]:
dof = cnl_results.data.nparam - nested_existing_results.data.nparam 
dof

2

The threshold value of the $\chi$-square test at 5% level is:

In [17]:
chi2.isf(0.05, dof)

5.991464547107983

Therefore, the null hypothesis can be rejected, and the cross-nested logit model is preferred. 